In [ ]:
%%capture
!pip install torch torchvision transformers datasets accelerate tqdm pillow

## CODE BASE

In [ ]:
from datasets import load_dataset
from transformers import AutoProcessor, SiglipModel, SiglipProcessor
import torch
from torch import nn, optim
from tqdm import tqdm
from PIL import Image
import random

from datasets import load_dataset
from transformers import AutoProcessor, SiglipModel
from torch import nn, optim
import torch
from tqdm import tqdm
from PIL import Image
from torch.utils.data import DataLoader
from torch.nn.functional import cosine_similarity


In [ ]:
#print(processed_dataset.column_names)

from huggingface_hub import login
login(token="")

In [ ]:
# ==============================================
# 🧩 STEP 3: Load dataset
# ==============================================
#dataset = load_dataset("EYEDOL/AGRILLAVA-image-text1")
#print(dataset)


from datasets import load_dataset, concatenate_datasets

# Define all your dataset names
dataset_names = [f"EYEDOL/AGRILLAVA-image-text{i}" for i in range(1, 16)]

# Load and collect all 'train' splits
all_splits = [load_dataset(name)["train"] for name in dataset_names]

# Merge them into one dataset
dataset = concatenate_datasets(all_splits)

print(dataset)
print(f"✅ Total samples after merge: {len(dataset)}")

print(dataset[0])


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/322 [00:00<?, ?B/s]

data/train-00000-of-00002.parquet:   0%|          | 0.00/338M [00:00<?, ?B/s]

data/train-00001-of-00002.parquet:   0%|          | 0.00/330M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/11671 [00:00<?, ? examples/s]

...

Generating train split:   0%|          | 0/11671 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/318 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/317M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/11671 [00:00<?, ? examples/s]

Dataset({
    features: ['image', 'text'],
    num_rows: 175065
})
✅ Total samples after merge: 175065
{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=185x500 at 0x7E9B18363D10>, 'text': 'The surface of the leaves in this picture is densely covered with reddish brown spots, with a considerable number and uneven distribution. This abnormal phenomenon is caused by Corn rust. Corn rust mainly occurs on maize leaves and can also infect leaf sheaths, stems, and bracts. In the early stage of infection, small pale yellow white spots appear on both sides of the leaves, with yellow halos around them. Later, they protrude to form yellow brown or even reddish brown blister spots, which are scattered or clustered in circular or elongated shapes, forming the summer spore pile of the pathogen. After the surface of the spore pile ruptures, rust like summer spores scatter. In the later stage, black blisters appear near or near the lesion, which is the winter spore pile of the pathogen

In [ ]:
def custom_collate_fn(batch):
    # Separate images and texts
    pixel_values = [item["pixel_values"] for item in batch]
    input_ids = [item["input_ids"] for item in batch]

    # Pad the input_ids to the maximum length in the batch
    max_len = max([len(ids) for ids in input_ids])
    padded_input_ids = [torch.cat([ids, torch.zeros(max_len - len(ids), dtype=torch.long)]) for ids in input_ids]

    # Stack the tensors
    pixel_values = torch.stack(pixel_values)
    input_ids = torch.stack(padded_input_ids)

    return {"pixel_values": pixel_values, "input_ids": input_ids}

In [ ]:
# ==============================================
# 🧩 STEP 4: Load SigLIP model and processor
# ==============================================
model_name = "google/siglip-base-patch16-224"
processor = AutoProcessor.from_pretrained(model_name)
model = SiglipModel.from_pretrained(model_name)

# ==============================================
# 🧩 STEP 5: Preprocess function
# ==============================================
def preprocess(examples):
    images = [img.convert("RGB") for img in examples["image"]]  # Already PIL
    texts = examples["text"]
    inputs = processor(images=images, text=texts, padding=True, truncation=True)
    return inputs

processed_dataset = dataset.map(
    preprocess,
    batched=True,
    remove_columns=dataset.column_names,
)


print("✅ Processed columns:", processed_dataset.column_names)

# ==============================================
# 🧩 STEP 6: Set format for PyTorch
# ==============================================
processed_dataset.set_format(type="torch", columns=["pixel_values", "input_ids"])

# ==============================================
# 🧩 STEP 7: Training setup
# ==============================================
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

optimizer = optim.AdamW(model.parameters(), lr=1e-5)
epochs = 3
batch_size = 16

train_loader = DataLoader(processed_dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate_fn)

# ==============================================
# 🏋️ STEP 8: Training loop with progress bar
# ==============================================
for epoch in range(epochs):
    model.train()
    total_loss = 0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")

    for batch in progress_bar:
        batch = {k: v.to(device) for k, v in batch.items()}  # Move tensors to GPU/CPU

        outputs = model(**batch)
        image_embeds = outputs.image_embeds
        text_embeds = outputs.text_embeds

        # Normalize
        image_embeds = image_embeds / image_embeds.norm(p=2, dim=-1, keepdim=True)
        text_embeds = text_embeds / text_embeds.norm(p=2, dim=-1, keepdim=True)

        # Contrastive loss
        logits = image_embeds @ text_embeds.t()
        labels = torch.arange(len(logits), device=device)
        loss = nn.CrossEntropyLoss()(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        progress_bar.set_postfix({"loss": f"{loss.item():.4f}"})

    print(f"Epoch {epoch+1} - Avg Loss: {total_loss/len(train_loader):.4f}")

# ==============================================
# 💾 STEP 9: Save model

preprocessor_config.json:   0%|          | 0.00/368 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/711 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/409 [00:00<?, ?B/s]

...

Epoch 1 - Avg Loss: 2.0012


Epoch 2/3: 100%|██████████| 10942/10942 [1:12:00<00:00,  2.53it/s, loss=1.5788]


Epoch 2 - Avg Loss: 1.9923


Epoch 3/3: 100%|██████████| 10942/10942 [1:12:11<00:00,  2.53it/s, loss=1.4494]

Epoch 3 - Avg Loss: 1.9905


In [ ]:
# ==============================================
model.save_pretrained("siglipFULL-agri-finetuned")
processor.save_pretrained("siglipFULL-agri-finetuned")
print("✅ Model saved to 'siglipFULL-agri-finetuned/'")

✅ Model saved to 'siglipFULL-agri-finetuned/'


In [ ]:
from huggingface_hub import HfApi, HfFolder

# Set your repo name (change if you prefer another)
repo_id = "EYEDOL/siglipFULL-agri-finetuned"

# Push both model and processor to Hugging Face Hub
model.push_to_hub(repo_id)
processor.push_to_hub(repo_id)

print(f"🚀 Model pushed to Hugging Face Hub at: https://huggingface.co/{repo_id}")